In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
file_path = "Diagonalization/diagonalizaton_results_"


# Diagonalization algorithm
df_diag_mixed = pd.read_csv(file_path + "Mixed_Diag.csv")
df_diag_PVs = pd.read_csv(file_path + "Fixed_PVs_Diag.csv")
df_diag_EVs = pd.read_csv(file_path + "Fixed_EVs_Diag.csv")

# AHE algorithm
df_ahe_mixed = pd.read_csv(file_path + "Mixed_AHE.csv")
df_ahe_PVs = pd.read_csv(file_path + "Fixed_PVs_AHE.csv")
df_ahe_EVs = pd.read_csv(file_path + "Fixed_EVs_AHE.csv")

#FP Algorithm
df_fp_mixed = pd.read_csv(file_path + "Mixed_FP.csv")
df_fp_PVs = pd.read_csv(file_path + "Fixed_PVs_FP.csv")
df_fp_EVs = pd.read_csv(file_path + "Fixed_EVs_FP.csv")

#SFP Algorithms
df_sfp_mixed = pd.read_csv(file_path + "Mixed_SFP.csv")
df_sfp_PVs = pd.read_csv(file_path + "Fixed_PVs_SFP.csv")
df_sfp_EVs = pd.read_csv(file_path + "Fixed_EVs_SFP.csv")


In [3]:
# add all dataframes 
df_all = [df_diag_mixed, df_diag_PVs, df_diag_EVs, 
          df_ahe_mixed,  df_ahe_PVs,  df_ahe_EVs, 
          df_fp_mixed,   df_fp_PVs,   df_fp_EVs, 
          df_sfp_mixed,  df_sfp_PVs,  df_sfp_EVs ]

algo_list = ["Diag", "AHE", "FP", "SFP"]

scen_list = ["Mixed", "Fixed_EVS", "Fixed_PVs"]

#****************************************************
cat_dict = dict()
for i in range(1,10):
    if i%2==0:
        cat_dict['DA'+str(i)] = "CAT_01"
    elif i%3==0:
        cat_dict['DA'+str(i)] = "CAT_02"
    else:
        cat_dict['DA'+str(i)] = "CAT_03"

#****************************************************
col_dict= dict()
for i in range(1,10):
    col_dict[str(i)] = "DA"+str(i)

In [5]:
for df in df_all:
    df.drop(df[df.Iteration==0].index, inplace=True)
    df.drop(columns=['Unnamed: 0'], inplace=True)

In [7]:
def change_width(ax, new_value) :
    for patch in ax.patches :
        current_width = patch.get_width()
        diff = current_width - new_value

        # we change the bar width
        patch.set_width(new_value)

        # we recenter the bar
        patch.set_x(patch.get_x() + diff * .5)

In [24]:
#**********************************************************
offer_col = ['offer_0'+str(x) for x in range(1,10)]
time_offer =['Time']
time_offer = [*offer_col, *time_offer]

#**********************************************************************
demand_col = ['bids_0'+str(x) for x in range(1,10)]
time_demand =['Time']
time_demand = [*demand_col, *time_demand]

# Offers, Demands  Last N Iterations Plot

In [36]:
counter = 0
counter_scen=0
demand = []
offer =[]
scenario=[]
time=[]

# how many last iterations to show
last_iterations = 50
no_iterations = 300

#iterate trough all dataframes and average their offers bids : all DAs in each 3 scenario
for df in df_all:
    no_iterations = len(df.groupby('Iteration').mean())
    if no_iterations > last_iterations:
        df = df[df['Iteration'] > no_iterations - last_iterations]
    temp_offer = df.groupby('Time').mean().drop(columns=['Iteration']).reset_index()[time_offer].drop('Time',axis=1).mean(axis=1)
    temp_demand = df.groupby('Time').mean().drop(columns=['Iteration']).reset_index()[time_demand].drop('Time',axis=1).mean(axis=1)
    count=16
    for i in range(len(temp_offer)):
        demand.append(temp_demand[i])
        offer.append(temp_offer[i])
        scenario.append(scen_list[counter%3])
        time.append(count)
        count+=1
    counter+=1
    if counter % 3 == 0:
        counter_scen+=1

# Create Dataframe from listed Values
df_compare = pd.DataFrame({'Demands':demand, "Offer":offer, 'Scenario':scenario, 'Time':time})

# Slice Dataframe for different plots


,Demands,Offer,Scenario,Time
25,0.128475,0.000000,Fixed_EVS,17
26,0.189283,0.000000,Fixed_EVS,18
27,0.220467,0.000000,Fixed_EVS,19
28,0.226999,0.000000,Fixed_EVS,20
29,0.701054,0.245878,Fixed_EVS,21
30,0.638970,0.221558,Fixed_EVS,22
31,0.792424,0.295252,Fixed_EVS,23
32,0.189129,0.000000,Fixed_EVS,24
33,0.134256,0.000000,Fixed_EVS,25
34,0.540575,0.284349,Fixed_EVS,26


In [23]:
time_demand, time_offer

(['bids_01',
  'bids_02',
  'bids_03',
  'bids_04',
  'bids_05',
  'bids_06',
  'bids_07',
  'bids_08',
  'bids_09',
  'Scenario'],
 ['offer_01',
  'offer_02',
  'offer_03',
  'offer_04',
  'offer_05',
  'offer_06',
  'offer_07',
  'offer_08',
  'offer_09',
  'Time'])